In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, TensorDataset
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [39]:
from sklearn.model_selection import train_test_split


# Set de datos
Antes de armar la red neuronal vamos a importar los datos que se van a requerir usando pandas. Vamos a hacer un preprocesamiento para que el formato sea compatible con la red neuronal que armaremos

In [40]:
PATH_DATA="C:/Users/alex_/Dropbox/Curso MinTIC/Proyectos Python/trading algo/datos/btc_full.xlsx"

In [41]:
datos = pd.read_excel(PATH_DATA)
datos.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume,MA20,MA150,MA20-E,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
0,2022-02-21 05:00:00,39315.492188,39394.437500,39242.855469,39242.855469,39242.855469,0,NaN,NaN,39242.855469,...,0,0,0,0,0,0,0,0,0,0
1,2022-02-21 06:00:00,39241.761719,39259.664062,39206.996094,39226.968750,39226.968750,0,NaN,NaN,39234.514941,...,0,0,0,0,0,0,0,0,0,0
2,2022-02-21 07:00:00,39229.574219,39291.769531,39125.968750,39291.769531,39291.769531,0,NaN,NaN,39255.538484,...,0,0,0,0,0,0,0,0,0,-100
3,2022-02-21 08:00:00,39289.988281,39347.812500,39119.832031,39119.832031,39119.832031,0,NaN,NaN,39216.362129,...,0,0,0,0,0,0,0,0,0,0
4,2022-02-21 09:00:00,39104.019531,39104.019531,38964.859375,38964.859375,38964.859375,0,NaN,NaN,39155.525753,...,0,0,0,0,0,0,0,0,0,0


In [42]:
datos.shape[0]
Datos_Min=int(datos.shape[0]*0.5)
Datos_Min
datos.shape[0]-Datos_Min

8750

In [43]:
guardar=datos.tail(3).copy()

In [44]:
datos=datos.loc[Datos_Min:]
datos.tail()

,Datetime,Open,High,Low,Close,Adj Close,Volume,MA20,MA150,MA20-E,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
17494,2024-02-21 01:00:00,52174.957031,52299.109375,52071.148438,52102.695312,52102.695312,8171520,51983.524023,51918.276901,52001.714746,...,0,0,0,0,0,0,0,0,0,0
17495,2024-02-21 02:00:00,52109.097656,52150.988281,52052.246094,52078.820312,52078.820312,111865856,51990.868945,51920.260313,52009.058133,...,0,0,0,0,0,0,0,0,0,0
17496,2024-02-21 03:00:00,52077.339844,52081.101562,52010.375000,52011.828125,52011.828125,125765632,52000.116992,51921.680625,52009.321942,...,0,0,0,0,0,0,0,0,0,0
17497,2024-02-21 04:00:00,52015.601562,52015.601562,51835.246094,51960.761719,51960.761719,159754240,52009.819531,51922.544714,52004.697159,...,0,0,0,0,0,0,0,0,0,0
17498,2024-02-21 05:00:00,51957.148438,52096.062500,51957.148438,51960.144531,51960.144531,80152576,52018.841602,51923.482240,52000.454052,...,0,0,0,0,0,0,0,0,0,0


In [45]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8750 entries, 8749 to 17498
Columns: 101 entries, Datetime to CDL_XSIDEGAP3METHODS
dtypes: datetime64[ns](1), float64(36), int64(64)
memory usage: 6.7 MB


In [46]:
for colum in datos.columns:
    print(colum)

Datetime
Open
High
Low
Close
Adj Close
Volume
MA20
MA150
MA20-E
MA150-E
Senal
Posicion
SMA-4
SMA-12
SMA-10
SMA-55
ADX
MD+
MD-
SQZ
macd
macd_signal
macd_hist
HA_Open
HA_High
HA_Low
HA_Close
stoch
stoch_signal
ATR
BB-B. Alta
BB-B. Baja
BB-B. Media
BB-Altura
BB-B. Alta1s
BB-B. Baja1s
BB-B. Media1s
BB-Altura1s
CDL_2CROWS
CDL_3BLACKCROWS
CDL_3INSIDE
CDL_3LINESTRIKE
CDL_3OUTSIDE
CDL_3STARSINSOUTH
CDL_3WHITESOLDIERS
CDL_ABANDONEDBABY
CDL_ADVANCEBLOCK
CDL_BELTHOLD
CDL_BREAKAWAY
CDL_CLOSINGMARUBOZU
CDL_CONCEALBABYSWALL
CDL_COUNTERATTACK
CDL_DARKCLOUDCOVER
CDL_DOJI_10_0.1
CDL_DOJISTAR
CDL_DRAGONFLYDOJI
CDL_ENGULFING
CDL_EVENINGDOJISTAR
CDL_EVENINGSTAR
CDL_GAPSIDESIDEWHITE
CDL_GRAVESTONEDOJI
CDL_HAMMER
CDL_HANGINGMAN
CDL_HARAMI
CDL_HARAMICROSS
CDL_HIGHWAVE
CDL_HIKKAKE
CDL_HIKKAKEMOD
CDL_HOMINGPIGEON
CDL_IDENTICAL3CROWS
CDL_INNECK
CDL_INSIDE
CDL_INVERTEDHAMMER
CDL_KICKING
CDL_KICKINGBYLENGTH
CDL_LADDERBOTTOM
CDL_LONGLEGGEDDOJI
CDL_LONGLINE
CDL_MARUBOZU
CDL_MATCHINGLOW
CDL_MATHOLD
CDL_MORNINGDOJIST

Adj Close Senal
Posicion

In [47]:
datos.iloc[0]["Open"]

24841.748046875

In [48]:
# Crear un DataFrame de ejemplo
data = {
    "Open": [10, 20, 30, 40, 50],
}

df = pd.DataFrame(data)

# Asignar los valores desplazados de la columna "Open" a la columna "Open+1"
df["Open+1"] = df["Open"].shift(-1)

print(df)

   Open  Open+1
0    10    20.0
1    20    30.0
2    30    40.0
3    40    50.0
4    50     NaN


In [49]:
datos["Y_OPEN"]=datos["Open"].shift(-1)
datos["Y_CLOSE"]=datos["Close"].shift(-1)
datos["Y_High"]=datos["High"].shift(-1)
datos["Y_Low"]=datos["Low"].shift(-1)
datos["Y_Volume"]=datos["Volume"].shift(-1)



In [50]:
datos[["Open","Y_OPEN"]]

,Open,Y_OPEN
8749,24841.748047,24821.121094
8750,24821.121094,24869.070312
8751,24869.070312,24820.677734
8752,24820.677734,24778.292969
8753,24778.292969,24736.238281
...,...,...
17494,52174.957031,52109.097656
17495,52109.097656,52077.339844
17496,52077.339844,52015.601562
17497,52015.601562,51957.148438


Imprimiento los primeros 5 elementos podemos ver que nuestra variable destino es la ultima columna **Exited** donde 1 es que se dio de baja y 0 que se quedó.

Es importante que en este paso vayamos pensando que tendremos que hacer con algunas columnas, ya sea eliminarlas, normalizarlas, llevar alguna transformación etc. Por ejemplo:
* RowNumber, CustomerID, Surname no nos dicen nada de utilidad. Predecir si alguien va a darse de baja por su ID de cliente, apellido o numero de renglon en la base de datos no nos va a dar buenos resultados. 
* Las columnas que son categoricas las vamos a transformar con **one hot encoding** (Genero y ubicacion geográfica)

In [51]:
datos= datos.dropna()

In [52]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8749 entries, 8749 to 17497
Columns: 106 entries, Datetime to Y_Volume
dtypes: datetime64[ns](1), float64(41), int64(64)
memory usage: 7.1 MB


In [53]:
# Separamos la ultima columna para que sea variable destino
datos_y = datos[["Y_OPEN","Y_CLOSE","Y_High","Y_Low", "Y_Volume"]]
datos_y.head()


,Y_OPEN,Y_CLOSE,Y_High,Y_Low,Y_Volume
8749,24821.121094,24865.910156,24868.482422,24821.121094,0.0
8750,24869.070312,24820.710938,24872.751953,24793.365234,0.0
8751,24820.677734,24787.539062,24826.080078,24786.083984,0.0
8752,24778.292969,24740.361328,24783.537109,24717.898438,4632576.0
8753,24736.238281,24825.718750,24825.718750,24721.765625,0.0


In [54]:
datos_y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8749 entries, 8749 to 17497
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Y_OPEN    8749 non-null   float64
 1   Y_CLOSE   8749 non-null   float64
 2   Y_High    8749 non-null   float64
 3   Y_Low     8749 non-null   float64
 4   Y_Volume  8749 non-null   float64
dtypes: float64(5)
memory usage: 410.1 KB


In [55]:
# Eliminamos las columnas que no funcionarán
datos_x = datos.drop(columns=["Datetime","Adj Close", "Senal","Posicion", "Y_OPEN", "Y_CLOSE", "Y_High","Y_Low", "Y_Volume"])
datos_x.head()

,Open,High,Low,Close,Volume,MA20,MA150,MA20-E,MA150-E,SMA-4,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
8749,24841.748047,24847.265625,24786.523438,24822.785156,0,24644.542676,24042.802083,24738.511129,24058.390999,24878.514648,...,0,0,0,0,0,0,0,0,0,0
8750,24821.121094,24868.482422,24821.121094,24865.910156,0,24670.782813,24063.412917,24750.644369,24069.086617,24857.127441,...,0,0,0,0,0,0,0,0,0,0
8751,24869.070312,24872.751953,24793.365234,24820.710938,0,24699.119727,24081.082669,24757.317376,24079.041906,24838.537598,...,0,0,0,0,0,0,0,0,0,0
8752,24820.677734,24826.080078,24786.083984,24787.539062,0,24726.586621,24099.427500,24760.195632,24088.425974,24824.236328,...,0,0,0,0,0,0,0,0,0,0
8753,24778.292969,24783.537109,24717.898438,24740.361328,4632576,24745.787109,24117.726693,24758.306650,24097.060880,24803.630371,...,0,0,0,0,0,0,0,0,0,0


In [56]:
for colum in datos_x.columns:
    print(colum)

Open
High
Low
Close
Volume
MA20
MA150
MA20-E
MA150-E
SMA-4
SMA-12
SMA-10
SMA-55
ADX
MD+
MD-
SQZ
macd
macd_signal
macd_hist
HA_Open
HA_High
HA_Low
HA_Close
stoch
stoch_signal
ATR
BB-B. Alta
BB-B. Baja
BB-B. Media
BB-Altura
BB-B. Alta1s
BB-B. Baja1s
BB-B. Media1s
BB-Altura1s
CDL_2CROWS
CDL_3BLACKCROWS
CDL_3INSIDE
CDL_3LINESTRIKE
CDL_3OUTSIDE
CDL_3STARSINSOUTH
CDL_3WHITESOLDIERS
CDL_ABANDONEDBABY
CDL_ADVANCEBLOCK
CDL_BELTHOLD
CDL_BREAKAWAY
CDL_CLOSINGMARUBOZU
CDL_CONCEALBABYSWALL
CDL_COUNTERATTACK
CDL_DARKCLOUDCOVER
CDL_DOJI_10_0.1
CDL_DOJISTAR
CDL_DRAGONFLYDOJI
CDL_ENGULFING
CDL_EVENINGDOJISTAR
CDL_EVENINGSTAR
CDL_GAPSIDESIDEWHITE
CDL_GRAVESTONEDOJI
CDL_HAMMER
CDL_HANGINGMAN
CDL_HARAMI
CDL_HARAMICROSS
CDL_HIGHWAVE
CDL_HIKKAKE
CDL_HIKKAKEMOD
CDL_HOMINGPIGEON
CDL_IDENTICAL3CROWS
CDL_INNECK
CDL_INSIDE
CDL_INVERTEDHAMMER
CDL_KICKING
CDL_KICKINGBYLENGTH
CDL_LADDERBOTTOM
CDL_LONGLEGGEDDOJI
CDL_LONGLINE
CDL_MARUBOZU
CDL_MATCHINGLOW
CDL_MATHOLD
CDL_MORNINGDOJISTAR
CDL_MORNINGSTAR
CDL_ONNECK
CDL_

# Convertimos en one hot encoding las columnas de genero y zona geográfica
datos_x = pd.get_dummies(datos_x)
datos_x.head()

Ya se cuenta con una variable que tiene todas las entradas al modelo **datos_x** y otra con la salida **datos_y**. Para datos_x se cuenta con una columna por cada categoria de las variables Genero y Zona geofráfica.


# Escalado datos
Ahora vamos a escalar los valores para que esten dentro de un rango mas corto.

In [57]:
datos_x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8749 entries, 8749 to 17497
Data columns (total 97 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Open                  8749 non-null   float64
 1   High                  8749 non-null   float64
 2   Low                   8749 non-null   float64
 3   Close                 8749 non-null   float64
 4   Volume                8749 non-null   int64  
 5   MA20                  8749 non-null   float64
 6   MA150                 8749 non-null   float64
 7   MA20-E                8749 non-null   float64
 8   MA150-E               8749 non-null   float64
 9   SMA-4                 8749 non-null   float64
 10  SMA-12                8749 non-null   float64
 11  SMA-10                8749 non-null   float64
 12  SMA-55                8749 non-null   float64
 13  ADX                   8749 non-null   float64
 14  MD+                   8749 non-null   float64
 15  MD-                   

## convertir las columnas en tensores


In [58]:
# Normalizar los datos
scaler = StandardScaler()
X = scaler.fit_transform(datos_x)

scalery=StandardScaler()
Y=scalery.fit_transform(datos_y)

In [59]:
X_tensor  = torch.tensor(X, dtype=torch.float32).cuda()
y_tensor  = torch.tensor(Y, dtype=torch.float32).cuda()

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

## clase modelo

class Modelo(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Modelo, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.fc5(out)
        return out

In [61]:
class Modelo(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Modelo, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        #self.fc4 = nn.Linear(hidden_size, hidden_size)
        #self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        #out = self.fc4(out)
        #out = self.relu4(out)
        out = self.fc5(out)
        return out

class Modelo(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Modelo, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, 500)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(500, 80)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(80, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.fc5(out)
        return out

In [62]:
input_size = X_train.shape[1]
output_size = y_train.shape[1]
hidden_size = 1000

In [63]:
model = Modelo(input_size, hidden_size, output_size).cuda()

In [64]:
import torch.optim as optim

In [65]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [66]:
list_loss=[]

In [67]:
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward pass y optimización
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Mostrar la pérdida en cada época
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.7f}')
    list_loss.append({"epoca":epoch,"loss":loss.item()})
    if loss.item()< 0.0000009:
        print("###########################detener##################################")
        break

# Evaluación del modelo
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Loss on test set: {test_loss.item():.7f}')
    

Epoch [1/10000], Loss: 1.0005895
Epoch [2/10000], Loss: 0.5972338
Epoch [3/10000], Loss: 0.2198389
Epoch [4/10000], Loss: 0.6830021
Epoch [5/10000], Loss: 0.2499190
Epoch [6/10000], Loss: 0.2171012
Epoch [7/10000], Loss: 0.3173317
Epoch [8/10000], Loss: 0.3365638
Epoch [9/10000], Loss: 0.2939422
Epoch [10/10000], Loss: 0.2302708
Epoch [11/10000], Loss: 0.1751761
Epoch [12/10000], Loss: 0.1527272
Epoch [13/10000], Loss: 0.1728123
Epoch [14/10000], Loss: 0.1981627
Epoch [15/10000], Loss: 0.1880942
Epoch [16/10000], Loss: 0.1579010
Epoch [17/10000], Loss: 0.1397108
Epoch [18/10000], Loss: 0.1430642
Epoch [19/10000], Loss: 0.1553931
Epoch [20/10000], Loss: 0.1608392
Epoch [21/10000], Loss: 0.1543179
Epoch [22/10000], Loss: 0.1409877
Epoch [23/10000], Loss: 0.1293587
Epoch [24/10000], Loss: 0.1253053
Epoch [25/10000], Loss: 0.1283510
Epoch [26/10000], Loss: 0.1316620
Epoch [27/10000], Loss: 0.1287740
Epoch [28/10000], Loss: 0.1208972
Epoch [29/10000], Loss: 0.1140409
Epoch [30/10000], Loss:

Epoch [121/10000], Loss: 0.0243666
Epoch [122/10000], Loss: 0.0232565
Epoch [123/10000], Loss: 0.0226723
Epoch [124/10000], Loss: 0.0227092
Epoch [125/10000], Loss: 0.0233105
Epoch [126/10000], Loss: 0.0243729
Epoch [127/10000], Loss: 0.0232724
Epoch [128/10000], Loss: 0.0229315
Epoch [129/10000], Loss: 0.0218968
Epoch [130/10000], Loss: 0.0210358
Epoch [131/10000], Loss: 0.0209680
Epoch [132/10000], Loss: 0.0213676
Epoch [133/10000], Loss: 0.0218030
Epoch [134/10000], Loss: 0.0219266
Epoch [135/10000], Loss: 0.0238871
Epoch [136/10000], Loss: 0.0226135
Epoch [137/10000], Loss: 0.0219466
Epoch [138/10000], Loss: 0.0205006
Epoch [139/10000], Loss: 0.0204439
Epoch [140/10000], Loss: 0.0196427
Epoch [141/10000], Loss: 0.0198745
Epoch [142/10000], Loss: 0.0214334
Epoch [143/10000], Loss: 0.0196757
Epoch [144/10000], Loss: 0.0188949
Epoch [145/10000], Loss: 0.0194867
Epoch [146/10000], Loss: 0.0204896
Epoch [147/10000], Loss: 0.0190776
Epoch [148/10000], Loss: 0.0194480
Epoch [149/10000], L

In [68]:
def r_squared(y_true, y_pred):
    y_mean = torch.mean(y_true)
    ss_tot = torch.sum((y_true - y_mean) ** 2)
    ss_res = torch.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

# Calcular R^2 en el conjunto de prueba
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    r2 = r_squared(y_test, test_outputs)
    print(f'R^2 on test set: {r2.item():.7f}')

R^2 on test set: 0.7840050


In [69]:
with torch.no_grad():
    model.eval()
    tsalida = model(X_tensor[-1])

In [70]:
tsalida

tensor([ 2.7463,  2.7425,  2.7491,  2.7615, -0.2226], device='cuda:0')

In [71]:
output_cpu = tsalida.cpu().numpy()
output_cpu

array([ 2.746266  ,  2.7424617 ,  2.7491298 ,  2.7614982 , -0.22261317],
      dtype=float32)

output_original_scale = scalery.inverse_transform(output_cpu)

In [72]:
with torch.no_grad():
    model.eval()
    tsalida = model(X_tensor[3100:3101, :])  # Selecciona una muestra de X_tensor
    output_cpu = tsalida.cpu().numpy()

# Asegúrate de que scalery sea el mismo objeto que usaste para escalar tus datos de salida antes de entrenar el modelo
output_original_scale = scalery.inverse_transform(output_cpu)

In [73]:
output_original_scale

array([[3.0448934e+04, 3.0495572e+04, 3.0548795e+04, 3.0445076e+04,
        7.8138816e+07]], dtype=float32)

In [74]:
datos.tail()

,Datetime,Open,High,Low,Close,Adj Close,Volume,MA20,MA150,MA20-E,...,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS,Y_OPEN,Y_CLOSE,Y_High,Y_Low,Y_Volume
17493,2024-02-21 00:00:00,52273.535156,52368.816406,52154.316406,52211.289062,52211.289062,0,51973.289062,51915.479010,51991.085213,...,0,0,0,0,0,52174.957031,52102.695312,52299.109375,52071.148438,8171520.0
17494,2024-02-21 01:00:00,52174.957031,52299.109375,52071.148438,52102.695312,52102.695312,8171520,51983.524023,51918.276901,52001.714746,...,0,0,0,0,0,52109.097656,52078.820312,52150.988281,52052.246094,111865856.0
17495,2024-02-21 02:00:00,52109.097656,52150.988281,52052.246094,52078.820312,52078.820312,111865856,51990.868945,51920.260313,52009.058133,...,0,0,0,0,0,52077.339844,52011.828125,52081.101562,52010.375000,125765632.0
17496,2024-02-21 03:00:00,52077.339844,52081.101562,52010.375000,52011.828125,52011.828125,125765632,52000.116992,51921.680625,52009.321942,...,0,0,0,0,0,52015.601562,51960.761719,52015.601562,51835.246094,159754240.0
17497,2024-02-21 04:00:00,52015.601562,52015.601562,51835.246094,51960.761719,51960.761719,159754240,52009.819531,51922.544714,52004.697159,...,0,0,0,0,0,51957.148438,51960.144531,52096.062500,51957.148438,80152576.0


In [75]:
datos.loc[3334:3336]

,Datetime,Open,High,Low,Close,Adj Close,Volume,MA20,MA150,MA20-E,...,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS,Y_OPEN,Y_CLOSE,Y_High,Y_Low,Y_Volume


In [76]:
x_prueba_t=scaler.transform(datos_x.loc[16000:16001, :])

In [77]:
datos_x.loc[16000:16001, :]

,Open,High,Low,Close,Volume,MA20,MA150,MA20-E,MA150-E,SMA-4,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
16000,44172.429688,44259.953125,43811.957031,43811.957031,605323264,43117.883594,42324.507839,43292.033760,42423.281168,43896.220703,...,0,0,0,0,0,0,0,0,0,100
16001,43803.019531,43803.019531,43269.132812,43489.601562,1120729088,43172.019141,42330.211328,43310.849741,42437.404617,43859.245117,...,0,0,0,0,0,0,0,0,0,0


In [78]:
datos_y.loc[16000:16001, :]

,Y_OPEN,Y_CLOSE,Y_High,Y_Low,Y_Volume
16000,43803.019531,43489.601562,43803.019531,43269.132812,1.120729e+09
16001,43441.500000,43421.804688,43699.824219,43380.578125,4.326953e+08


In [79]:
X_tensor_prueba  = torch.tensor(x_prueba_t, dtype=torch.float32).cuda()

In [80]:
with torch.no_grad():
    model.eval()
    y_tensor_salida = model(X_tensor_prueba)
    output_cpu = y_tensor_salida.cpu().numpy()

In [81]:
output_cpu


array([[1.6194632, 1.5785894, 1.6025213, 1.5628371, 1.2207962],
       [1.5688422, 1.5707612, 1.5901188, 1.5774058, 0.2684143]],
      dtype=float32)

In [82]:
y_predicido = scalery.inverse_transform(output_cpu)
y_predicido

array([[4.3815418e+04, 4.3527289e+04, 4.3819582e+04, 4.3283789e+04,
        1.0903661e+09],
       [4.3449328e+04, 4.3470652e+04, 4.3729535e+04, 4.3388762e+04,
        3.9920237e+08]], dtype=float32)

In [83]:
scalery.feature_names_in_

array(['Y_OPEN', 'Y_CLOSE', 'Y_High', 'Y_Low', 'Y_Volume'], dtype=object)

In [84]:
df_predic = pd.DataFrame(y_predicido)
df_predic.columns=scalery.feature_names_in_
df_predic

,Y_OPEN,Y_CLOSE,Y_High,Y_Low,Y_Volume
0,43815.417969,43527.289062,43819.582031,43283.789062,1.090366e+09
1,43449.328125,43470.652344,43729.535156,43388.761719,3.992024e+08


In [85]:
datos_y.loc[3334:3336, :]

,Y_OPEN,Y_CLOSE,Y_High,Y_Low,Y_Volume


In [86]:
guardar

,Datetime,Open,High,Low,Close,Adj Close,Volume,MA20,MA150,MA20-E,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
17496,2024-02-21 03:00:00,52077.339844,52081.101562,52010.375000,52011.828125,52011.828125,125765632,52000.116992,51921.680625,52009.321942,...,0,0,0,0,0,0,0,0,0,0
17497,2024-02-21 04:00:00,52015.601562,52015.601562,51835.246094,51960.761719,51960.761719,159754240,52009.819531,51922.544714,52004.697159,...,0,0,0,0,0,0,0,0,0,0
17498,2024-02-21 05:00:00,51957.148438,52096.062500,51957.148438,51960.144531,51960.144531,80152576,52018.841602,51923.482240,52000.454052,...,0,0,0,0,0,0,0,0,0,0


In [87]:
def predicion_de_precio(model, escaladorX, escaladorY, dataframe=pd.DataFrame):
    #prediciones=1
    #while prediciones<=5: 
    datos_x = dataframe.drop(columns=["Datetime","Adj Close", "Senal","Posicion"])
    X = escaladorX.transform(datos_x)
    X_tensor_a_predecir  = torch.tensor(X, dtype=torch.float32).cuda()
    with torch.no_grad():
        model.eval()
        y_tensor_salida = model(X_tensor_a_predecir)
        output_cpu = y_tensor_salida.cpu().numpy()
    y_predicido = escaladorY.inverse_transform(output_cpu)
    df_predic = pd.DataFrame(y_predicido)
    df_predic.columns=escaladorY.feature_names_in_
    return df_predic


In [88]:
df_loss=pd.DataFrame(list_loss)

In [89]:
df_loss

,epoca,loss
0,0,1.000589
1,1,0.597234
2,2,0.219839
3,3,0.683002
4,4,0.249919
...,...,...
9995,9995,0.001015
9996,9996,0.000485
9997,9997,0.000368
9998,9998,0.000515


In [90]:
df_loss.min()

epoca    0.000000
loss     0.000002
dtype: float64

In [91]:
DF_Predecido=predicion_de_precio(model,scaler,scalery,guardar.loc[17497:17498])
DF_Predecido

,Y_OPEN,Y_CLOSE,Y_High,Y_Low,Y_Volume
0,51964.472656,51947.738281,52144.078125,51920.414062,42853372.0
1,52477.714844,52487.437500,52524.457031,52510.132812,589314496.0


In [92]:
guardar

,Datetime,Open,High,Low,Close,Adj Close,Volume,MA20,MA150,MA20-E,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
17496,2024-02-21 03:00:00,52077.339844,52081.101562,52010.375000,52011.828125,52011.828125,125765632,52000.116992,51921.680625,52009.321942,...,0,0,0,0,0,0,0,0,0,0
17497,2024-02-21 04:00:00,52015.601562,52015.601562,51835.246094,51960.761719,51960.761719,159754240,52009.819531,51922.544714,52004.697159,...,0,0,0,0,0,0,0,0,0,0
17498,2024-02-21 05:00:00,51957.148438,52096.062500,51957.148438,51960.144531,51960.144531,80152576,52018.841602,51923.482240,52000.454052,...,0,0,0,0,0,0,0,0,0,0


# no funciona

escalador = StandardScaler()
datos_x = escalador.fit_transform(datos_x)
print(datos_x)

# Dividir datos entre entrenamiento y test

datos_x.shape[0]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(datos_x, datos_y, test_size = 0.2, random_state = 2)

print("X Train: {}, X Test: {}, y_train: {}, y_test: {}".format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

n_entradas = X_train.shape[1]

n_entradas

torch.cuda.is_available()

# Tensores
Para poder procesar los datos en la red neuronal necesitamos que todos los datos estén en tensores, asi que haremos las conversiones necesarias

t_X_train = torch.from_numpy(X_train).float().to("cuda") #MPS
t_X_test = torch.from_numpy(X_test).float().to("cuda")
t_y_train = torch.from_numpy(y_train.values).float().to("cuda")
t_y_test = torch.from_numpy(y_test.values).float().to("cuda")
t_y_train = t_y_train[:,None]
t_y_test = t_y_test[:, None]


test = TensorDataset(t_X_test, t_y_test)
print(test[0])

t_y_train

t_y_test

n_entradas

# Estructura de la red neuronal
Ahora vamos a armar una estructura básica de una red neuronal la cual va a recibir los datos de **X** para eventualmente poder predecir **y**

Para hacer esto tenemos que crear una Clase la cual hereda de nn.Module de torch.

class Red(nn.Module):
    
    def __init__(self, n_entradas):
        super(Red, self).__init__()
        self.linear1 = nn.Linear(n_entradas, 97)
        self.linear2 = nn.Linear(97, 200)
        self.linear3 = nn.Linear(200,300)
        self.linear4 = nn.Linear(300, 50)
        self.linear5 = nn.Linear(50, 8)
        self.linear6 = nn.Linear(8, 1)
    
    def forward(self, inputs):
        pred_1 = torch.sigmoid(input=self.linear1(inputs))
        pred_2 = torch.sigmoid(input=self.linear2(pred_1))
        prediction3 = torch.sigmoid(input=self.linear3(pred_2))
        prediction4 = torch.sigmoid(input=self.linear4(prediction3))
        prediction5 = torch.sigmoid(input=self.linear5(prediction4))
        pred_f = torch.sigmoid(input=self.linear6(prediction5))
        return pred_f


# Definir la arquitectura de la red neuronal

import torch.optim as optim
class Modelo(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Modelo, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 80)  
        self.fc3 = nn.Linear(80, 5)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Definir las dimensiones de la entrada, salida y capas ocultas
input_size = 97
hidden_size = 300
output_size = 5

# Crear una instancia del modelo
model = Modelo(input_size, hidden_size, output_size)

# Definir la función de pérdida (Mean Squared Error) y el optimizador (por ejemplo, Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Se puede ajustar la tasa de aprendizaje según sea necesario

# Ejemplo de cómo entrenar la red
# Suponiendo que tienes datos de entrada X_train y etiquetas de salida y_train
# (asegúrate de convertirlos a tensores de PyTorch)
X_train = torch.randn(3000, 97)
y_train = torch.randn(3000, 5)

# Número de épocas de entrenamiento
num_epochs = 10

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward pass y optimización
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Mostrar la pérdida en cada época
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

t_X_train.shape

t_y_test[0]

t_y_train = torch.squeeze(t_y_train, dim=1)
t_y_test = torch.squeeze(t_y_test, dim=1)

n_entradas

%%time
lr = 0.001
epochs = 5000
estatus_print = 100

model = Red(n_entradas=n_entradas).to('cuda')
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
print("Arquitectura del modelo: {}".format(model))
historico = pd.DataFrame()

print("Entranando el modelo")
for epoch in range(1, epochs+1):
    y_pred= model(t_X_train)
    print("Entranando el modelo2")
    loss = loss_fn(input=y_pred, target=t_y_train)
    print("Entranando el modelo3")
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % estatus_print == 0:
        print(f"\nEpoch {epoch} \t Loss: {round(loss.item(), 4)}")
    
    with torch.no_grad():
        y_pred = model(t_X_test)
        y_pred_class = y_pred.round()
        correct = (y_pred_class == t_y_test).sum()
        accuracy = 100 * correct / float(len(t_y_test))
        if epoch % estatus_print == 0:
            print("Accuracy: {}".format(accuracy.item()))
    
    df_tmp = pd.DataFrame(data={
        'Epoch': epoch,
        'Loss': round(loss.item(), 4),
        'Accuracy': round(accuracy.item(), 4)
    }, index=[0])
    historico = pd.concat(objs=[historico, df_tmp], ignore_index=True, sort=False)

print("Accuracy final: {}".format(round(accuracy.item(), 4)))

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10, 10))
plt.plot(historico['Epoch'], historico['Loss'], label='Loss')
plt.title("Loss", fontsize=25)
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Loss", fontsize=12)
plt.grid()
plt.show()

plt.figure(figsize=(10, 10))
plt.plot(historico['Epoch'], historico['Accuracy'], label='Accuracy')
plt.title("Accuracy", fontsize=25)
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)
plt.grid()
plt.show()

t_X_test[5]

prediccion = model(t_X_test[5])
print(prediccion)

t_y_test[5]